In [ ]:
import numpy as np
import pandas as pd

# Test word vectors

In [5]:
wvec = '/Users/pabloherrero/Downloads/cc.he.300.bin'
# with open(wvec, 'rb') as f:
#     lines = f.readlines()
#     print(len(lines))
#     # print(lines[0].decode('utf-8'))

In [17]:
import spacy

In [18]:
vec_path = '/Users/pabloherrero/Documents/ManHatTan/notebooks/he_vectors'
nlp = spacy.load(vec_path)

In [19]:
doc = nlp("זהו טקסט בעברית.")
for token in doc:
    print(token.text, token.has_vector, token.vector_norm)

זהו True 1.0681664
טקסט True 0.8047567
בעברית True 0.40524793
. True 1.5317025


In [20]:
doc = nlp("חברה שלי פיפו לא אדום.")
for token in doc:
    print(token.text, token.has_vector, token.vector_norm)

חברה True 0.77420473
שלי True 1.0418928
פיפו True 0.0
לא True 1.9327171
אדום True 0.83313257
. True 1.5317025


In [21]:
import numpy as np

In [22]:
adom_id = nlp.vocab.strings['אדום']
adom_vec = nlp.vocab.vectors[adom_id]
most_similar_words = nlp.vocab.vectors.most_similar(np.asarray([adom_vec]), n=5)
words = [nlp.vocab.strings[w] for w in most_similar_words[0][0]]
words


['אדום', 'כחול', 'צהוב', 'ירוק', 'שחור']

In [ ]:
from bidi.algorithm import get_display


In [57]:
selected_word = 'להגיע'
# selected_word = get_display(selected_word)
selected_word_id = nlp.vocab.strings['להגיע']
sel_word_vec = nlp.vocab.vectors[selected_word_id]
most_similar_words = nlp.vocab.vectors.most_similar(np.asarray([sel_word_vec]), n=5)
words = [nlp.vocab.strings[w] for w in most_similar_words[0][0]]

filtered_words = []
for w in words:
    if selected_word in w:
        print('Contained in original: ', w)
        continue
    filtered_words.append(w)
filtered_words

Contained in original:  להגיע
Contained in original:  ולהגיע


['להיכנס', 'לגשת', 'להתקרב']

In [ ]:
adom_id = nlp.vocab.strings['אמרתי']
adom_vec = nlp.vocab.vectors[adom_id]
most_similar_words = nlp.vocab.vectors.most_similar(np.asarray([adom_vec]), n=8)
words = [nlp.vocab.strings[w] for w in most_similar_words[0][0]]
words

['אמרתי', 'ואמרתי', 'אגיד', 'חשבתי', 'ידעתי', 'שאמרתי', 'אומר', 'אמר']

In [15]:
lippath = '/Users/pabloherrero/Documents/ManHatTan/data/processed/LIPSTICK/hebrew_db.lip'
lip = pd.read_csv(lippath)
lip.set_index('word_ll', inplace=True, drop=False)

In [157]:

# Load your spaCy model that contains the Hebrew vectors
nlp = spacy.load("he_vectors")

# Define the target word and process it to get its token (make sure it has a vector)
target_word = "עכביש"
target_token = nlp(target_word)[0]

other_words = lip.word_ll.values.tolist()  # List of words to compare with the target word
# Compute similarity for each word in the list
similarities = []
for word in other_words:
    token = nlp(word)[0]
    # Check if both tokens have vectors
    if target_token.has_vector and token.has_vector:
        sim = target_token.similarity(token)
        similarities.append((word, sim))
    else:
        similarities.append((word, 0.0))

# Sort words by similarity in descending order
similarities.sort(key=lambda x: x[1], reverse=True)

# Print the top 10 most similar words
for word, score in similarities[:10]:
    print(f"{word}: {score:.3f}, {lip.loc[word, 'word_ul']}")


עכביש: 1.000, spider
קוף: 0.344, monkey
מפלצת: 0.316, monster
נחש: 0.286, snake
יצור: 0.260, creature
מכשפה: 0.259, witch
קוסם: 0.257, magician
מיקרופון: 0.254, microphone
טיפש: 0.251, Stupid
שריון: 0.251, armor


In [166]:
len(other_words)

288

In [167]:
all_entries = lip.word_ll.values.tolist()
vect_word = []
vectors = []
for word in all_entries:
    token = nlp(word)[0]
    # Check if both tokens have vectors
    if target_token.has_vector and token.has_vector:
        vect_word.append(word)
        vectors.append(token.vector)

In [ ]:
pathout = '/Users/pabloherrero/Documents/ManHatTan/data/processed/vectors_lip/vectors_heb_lip.npz'

np.savez(pathout, tokens=vect_word, vectors=vectors)

In [178]:
data = np.load(pathout)
tokens = data['tokens']
vectors = data['vectors']
token_to_vector = {token: vector for token, vector in zip(vect_word, vectors)}


In [190]:
specific_word = 'עכביש'
specific_vector = token_to_vector[specific_word]
similarities = []
norm = np.dot(specific_vector, specific_vector)
for word in tokens:
    token = token_to_vector[word]
    sim = np.dot(specific_vector, token) / norm
    similarities.append((word, sim))
similarities.sort(key=lambda x: x[1], reverse=True)
# Print the top 10 most similar words
for word, score in similarities[1:4]:
    print(f"{word}: {score:.3f}, {lip.loc[word, 'word_ul']}")

קוף: 0.659, monkey
חץ: 0.574, arrow
נחש: 0.503, snake


# Functions to sample similar words

In [200]:
from random import sample

def load_similar_words(pathout, target_word):
    word_vectors = np.load(pathout)
    tokens = word_vectors['tokens']
    vectors = word_vectors['vectors']
    token_to_vector = {token: vector for token, vector in zip(tokens, vectors)}
    try:
        target_vector = token_to_vector[target_word]
    except IndexError as e:
        print(e, f' with {target_word}')
        return
    similarities = []

    norm = np.dot(target_vector, target_vector)
    for word in tokens:
        token = token_to_vector[word]
        sim = np.dot(target_vector, token) / norm
        similarities.append((word, sim))
    similarities.sort(key=lambda x: x[1], reverse=True)
    # return the top 3 most similar words
    return [word for word, _ in similarities[1:10]]

def sample_similar_options(lipstick_path : str, iquest : int, modality : str, n_options : int = 3):

    if modality == 'dt': word_lang = 'word_ul'
    elif modality == 'rt': word_lang = 'word_ll'
    else: print('Incorrect modality in rnd_options function')
    lip = pd.read_csv(lipstick_path)
    target_word = lip.iloc[iquest]['word_ll']
    lip = lip.set_index('word_ll', drop=False)
    options = {}
    #### Change this: to be extracted from lipstick_path:
    vector_path = '/Users/pabloherrero/Documents/ManHatTan/data/processed/vectors_lip/vectors_heb_lip.npz'
    similar_words = load_similar_words(vector_path, target_word=target_word)
    
    rnd_similar_words = sample(similar_words, n_options)
    print(rnd_similar_words)
    for i in range(n_options):
        rndOp = lip.loc[rnd_similar_words][word_lang]
        print(rndOp)
        # options[rndOp] = False
    return options


In [199]:
vector_path = '/Users/pabloherrero/Documents/ManHatTan/data/processed/vectors_lip/vectors_heb_lip.npz'
load_similar_words(pathout=vector_path, target_word='עכביש')

['קוף', 'חץ', 'נחש', 'חכם', 'פשע', 'פחד', 'תת קרקעי', 'קוסם', 'מה אתה מסתכל']

In [201]:
iquest = 4
sample_similar_options(lipstick_path=lippath, iquest=iquest, modality='dt', n_options=4)

['נקי', 'מה נראה לך', 'רחב', 'יבש']
word_ll
נקי                         net
מה נראה לך    What do you think
רחב                        wide
יבש                         dry
Name: word_ul, dtype: object
word_ll
נקי                         net
מה נראה לך    What do you think
רחב                        wide
יבש                         dry
Name: word_ul, dtype: object
word_ll
נקי                         net
מה נראה לך    What do you think
רחב                        wide
יבש                         dry
Name: word_ul, dtype: object
word_ll
נקי                         net
מה נראה לך    What do you think
רחב                        wide
יבש                         dry
Name: word_ul, dtype: object


{}

# Filtering out-of-bags words

In [ ]:
selected_word = 'להגיע'
# selected_word = get_display(selected_word)
selected_word_id = nlp.vocab.strings['להגיע']
sel_word_vec = nlp.vocab.vectors[selected_word_id]
most_similar_words = nlp.vocab.vectors.most_similar(np.asarray([sel_word_vec]), n=5)
words = [nlp.vocab.strings[w] for w in most_similar_words[0][0]]

filtered_words = []
for w in words:
    if selected_word in w:
        print('Contained in original: ', w)
        continue
    filtered_words.append(w)
filtered_words

Contained in original:  להגיע
Contained in original:  ולהגיע


['להיכנס', 'לגשת', 'להתקרב']

In [154]:
def filter_sofits(word):
    from hebrew import FINAL_MINOR_LETTER_MAPPINGS as sofit_dict
    final_letter = word[-1]
    if final_letter in sofit_dict.keys():
        filtered_word = word[:-1] + sofit_dict[final_letter]
        return filtered_word
    else:
        return word

In [156]:
selected_word = 'ציון'
# selected_word = get_display(selected_word)
selected_word_id = nlp.vocab.strings[selected_word]
sel_word_vec = nlp.vocab.vectors[selected_word_id]
most_similar_words = nlp.vocab.vectors.most_similar(np.asarray([sel_word_vec]), n=20)
words = [nlp.vocab.strings[w] for w in most_similar_words[0][0]]
selected_filtered = filter_sofits(selected_word)
filtered_words = []
for w in words:
    wf = filter_sofits(w)
    if selected_filtered in wf:
        print('Contained in original: ', w)
        continue
    filtered_words.append(w)
filtered_words

Contained in original:  ציון
Contained in original:  הציון
Contained in original:  בציון
Contained in original:  מציון
Contained in original:  בן-ציון
Contained in original:  לציון
Contained in original:  ציוני
Contained in original:  ציונים
Contained in original:  הציונים


['בוויקינתוניםערכים',
 '1809-443-836החל',
 'ממוצע',
 'דינור',
 'יהונתן',
 'משה',
 'דוד',
 'יובל',
 'אברהם',
 'הזיתים',
 'הלברשטאם']